In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from prophet.serialize import model_from_json

In [ ]:
def query_yf(stock: tuple, period: int) -> pd.DataFrame:
    """
    Query the Yahoo Finance API for a given stock and store the data.

    :param stock: tuple
        A tuple containing three elements:
        - company (str): The name of the company.
        - ticker (str): The stock ticker symbol of the company.
        - exchange (str): The stock exchange where the company is listed.
    :param period: int
        The number of days of data to retrieve.

    :return: data: a pandas dataframe of the data
    """
    try:
        # Extract company, ticker, and exchange from the stock tuple
        company, ticker, exchange = stock

        yesterday = datetime.now() - timedelta(days=period)
        today = datetime.now()

        yesterday_str = yesterday.strftime('%Y-%m-%d')
        today_str = today.strftime('%Y-%m-%d')
        today_str = '2023-12-18'

        # Query the API using yfinance
        stock_data = yf.Ticker(ticker)
        data = stock_data.history(period='1d', start=yesterday_str, end=today_str)

        # Add the company name to the dataframe
        data['Company'] = company
        data['Ticker'] = ticker
        data['Exchange'] = exchange

        # Reorder the columns
        data = data[['Company', 'Ticker', 'Exchange', 'Open', 'High', 'Low', 'Close', 'Volume']]

        # Convert DataFrame index to a 'DatetimeIndex' without time zone information
        data.index = data.index.tz_localize(None)  # this is necessary for some algorithms

        return data
    except Exception as e:
        print(f"Error retrieving data: {e}")
        return None

In [ ]:
def query_yf_list(stocks: pd.DataFrame, period: int) -> pd.DataFrame:
    """
    Query the Yahoo Finance API for all stocks in a list based on their exchange.

    :param stocks: list
        A list of tuples containing three elements:
        - company (str): The name of the company.
        - ticker (str): The stock ticker symbol of the company.
        - exchange (str): The stock exchange where the company is listed.
    :param period: int
        The number of days of data to retrieve.

    :return: data: a list of pandas dataframes of the data
    """
    # Create an empty list
    data = []

    # Loop through the list of stocks
    for _, row in stocks.iterrows():
        stock = (row['Company'], row['Ticker'], row['Stock Exchange'])

        # Query the API using yfinance
        stock_data = query_yf(stock, period)

        # Append the data to the dataframe
        if stock_data is not None:
            data.append(stock_data)

    return data

In [ ]:
def day_average_std(df: pd.DataFrame, columns: [str] = ['Close', 'Open', 'High', 'Low', 'Volume'], days: int = 30, company_name: str = None) -> pd.DataFrame:
    """
    Compute the n-day average and standard deviation of the stock price.
    :param column: column to average
    :param df: dataframe of stock data
    :param days: number of days to average
    :return: dataframe of stock data with added column for an n day average
    """
    for column in columns:
        if company_name is not None:
            column_title = f"{company_name}-{column} Day Avg"
        else:
            column_title = f"{column} Day Avg"

        df[column_title] = df[column].rolling(days).mean().shift(1).astype('float32')  # shift to not include today
        day_mean = df[column].iloc[0:days+1].mean().astype('float32')
        df[column_title] = df[column_title].fillna(day_mean)

        # now add the standard deviation
        if company_name is not None:
            column_title = f"{company_name}-{column} Day Std"
        else:
            column_title = f"{column} Day Std"
        df[column_title] = df[column].rolling(days).std().shift(1).astype('float32')  # shift to not include today
        day_std = df[column].iloc[0:days+1].std().astype('float32')
        df[column_title] = df[column_title].fillna(day_std)

    return df

In [ ]:
def overall_averages(data_list: [pd.DataFrame]) -> pd.DataFrame:
    """
    Compute the average values for each column for all stocks in the list,
    corresponding to the previous day's averages.
    :param data_list: list of dataframes of individual stock data.
    :return: dataframe of overall averages.
    """
    concatenated_df = pd.concat(data_list)
    concatenated_df.drop(columns=['Company', 'Ticker', 'Exchange'], inplace=True)
    concatenated_df.set_index('Date', inplace=True)

    # Calculate daily averages
    daily_averages = concatenated_df.groupby(concatenated_df.index).mean().astype('float32')

    # Shift the averages to represent the previous day
    daily_averages = daily_averages.shift(1).astype('float32')

    # Rename columns to reflect they are lagged values
    daily_averages.columns = ['All Prev Day Avg ' + col for col in daily_averages.columns]
    daily_averages.drop(columns=['All Prev Day Avg index'], inplace=True)

    return daily_averages


In [ ]:
def exchange_averages(data_list: [pd.DataFrame]) -> [pd.DataFrame]:
    """
    Compute the average values for each column for all stocks in the exchange,
    corresponding to the previous day's averages.
    :param data_list: list of dataframes of individual stock data.
    :return: dictionary of dataframes of exchange averages.
    """
    concatenated_df = pd.concat(data_list)

    # Group by 'Date' and 'Exchange'
    grouped = concatenated_df.groupby(['Date', 'Exchange'])

    # Calculate the average of the required columns
    averages = grouped[['Open', 'High', 'Low', 'Close', 'Volume']].mean().astype('float32')

    # Shift the averages to represent the previous day
    averages = averages.groupby(level='Exchange').shift(1).astype('float32')

    # Rename columns to reflect they are lagged values
    averages.columns = ['Exchange Prev Day Avg ' + col for col in averages.columns]

    # Create a dictionary of dataframes for each exchange
    exchange_dfs = {exchange: df.reset_index() for exchange, df in averages.groupby(level='Exchange')}

    return exchange_dfs


In [ ]:
def add_exchange_averages_to_stocks(data_list: [pd.DataFrame], exchange_dfs: dict) -> [pd.DataFrame]:
    """
    Add exchange averages to each stock in the data list.

    :param data_list: list of dataframes of individual stock data.
    :param exchange_dfs: dictionary of dataframes containing exchange averages.
    :return: list of dataframes with added exchange average columns.
    """
    updated_data_list = []

    for stock_df in data_list:
        exchange_name = stock_df['Exchange'].iloc[0]

        # Get the corresponding exchange average dataframe
        exchange_avg_df = exchange_dfs.get(exchange_name, None)

        updated_df = pd.merge(stock_df, exchange_avg_df, how='left', on=['Date', 'Exchange'])
        updated_data_list.append(updated_df)

    return updated_data_list

In [ ]:
def add_overall_averages_to_stocks(data_list: [pd.DataFrame], average_dfs: [pd.DataFrame]):
    """
    Add overall averages to each stock in the data list.

    :param data_list: list of dataframes of individual stock data.
    :param average_dfs: list of dataframes containing overall averages.
    :return: list of dataframes with added overall average columns.
    """
    updated_data_list = []

    for stock_df in data_list:
        updated_df = pd.merge(stock_df, average_dfs, how='left', on='Date')
        updated_data_list.append(updated_df)

    return updated_data_list

In [ ]:
# start by loading the prophet models for each stock from the models directory
# load the past thirty days of data for each stock and process as necessary
# generate predictions for the next day
# place orders for the next day

In [ ]:
# load the prophet models for each stock from the models directory
stocks_list = pd.read_csv('Uranium Company Master List.csv')
models = {}
for stock in stocks_list:
    company_name = stock['Company']
    with open(f'models/{company_name}.json', 'r') as fin:
        models[company_name] = model_from_json(fin.read())


In [ ]:
# retrieve necessary data for predicting the next day and process as necessary
data_list = query_yf_list(stocks_list, 30)
for data in data_list:
    data.reset_index(inplace=True)  # integer index

exchange_averages_dfs = exchange_averages(data_list)
overall_averages_dfs = overall_averages(data_list)
data_list = add_exchange_averages_to_stocks(data_list, exchange_averages_dfs)
data_list = add_overall_averages_to_stocks(data_list, overall_averages_dfs)
for data in data_list:
    data = day_average_std(data)

In [ ]:
predictions = []

for stock_df in data_list:
    company_name = stock_df['Company'].iloc[0]
    ticker = stock_df['Ticker'].iloc[0]
    exchange = stock_df['Exchange'].iloc[0]

    model = models.get(company_name)
    if not model:
        continue  # or handle missing model for a company

    # Prepare the dataframe for Prophet prediction
    df_for_prediction = stock_df.copy()
    df_for_prediction['ds'] = df_for_prediction['Date']
    df_for_prediction.drop(columns=['Company', 'Ticker', 'Exchange', 'Date'], inplace=True)

    # Ensure current day's close value is available
    current_day_value = df_for_prediction['Close'].iloc[-1]  # Get the most recent closing value

    future = model.make_future_dataframe(periods=1)
    forecast = model.predict(df_for_prediction)

    # Extract the prediction for the next day
    next_day_prediction = forecast.iloc[-1]['yhat']  # Predicted Close value for the next day

    predictions.append((ticker, exchange, next_day_prediction, current_day_value))


In [ ]:
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId, ticker, exchange, next_day_prediction, current_day_value):
        # create contract
        contract = Contract()
        contract.symbol = ticker
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.primaryExchange = exchange

        # create order
        order = Order()
        # set action based on Prophet prediction
        if next_day_prediction > current_day_value:
            order.action = "SELL"
        elif next_day_prediction < current_day_value:
            order.action = "BUY"

       # using a set quantity, but could be changed with more complex logic based on
        # risk and etc
        order.totalQuantity = 10
        order.orderType = "MKT"

        # place order
        self.placeOrder(orderId, contract, order)

def main():
    app = TestApp()
    app.connect("127.0.0.1", 7497, 0)
    app.run()

if __name__ == "__main__":
    main()
